In [60]:
import unittest
import numpy as np
import importlib
import sys
sys.path.append('../')
from src import basis_class

# Reload the module to apply changes
importlib.reload(basis_class)
from src.basis_class import Basis

In [92]:
class BasisTestCase(unittest.TestCase):
    
    def test_get_correct_dimensions(self):
        n = 2
        basis_class = Basis(2)
        result = basis_class.dimension
        self.assertEqual(result, n)
    
    def test_only_even_dimensions(self):
        n = 3
        with self.assertRaises(Exception) as context:
            Basis(n)  # This should raise the exception

        
        self.assertEqual(str(context.exception), "Only positive even integers allowed!")

    def test_only_positive_dimensions(self):
        n = -4
        with self.assertRaises(Exception) as context:
            Basis(n)
        
        self.assertEqual(str(context.exception), "Only positive even integers allowed!")
        
    def test_only_integer_dimensions(self):
        n=3.45
        with self.assertRaises(Exception) as context:
            Basis(n)
        
        self.assertEqual(str(context.exception), "Only positive even integers allowed!")
      
      
    def test_creating_vector_of_correct_size(self):
        
        n= 4
        index = 1 
        basis_class = Basis(n)

        result = len(basis_class.create_basis_vectors(index))
        self.assertEqual(result, n)        

    def test_creates_unit_vector_correctly(self):

        n = 4
        basis_class = Basis(n)
        index = 2
        
        vector = basis_class.create_basis_vectors(index)

        self.assertEqual(index, np.nonzero(vector)[0])
        self.assertEqual(1, vector[index])
    
    def test_create_basis_vector_validation(self):
        n = 4
        index = -3
        with self.assertRaises(Exception) as context:
            basis_class = Basis(n)
            basis_class.create_basis_vectors(index)

        self.assertEqual(str(context.exception), "Index can't be less than zero!")

    def test_create_single_symmetric_matrix_is_correct(self):
            n = 2
            first_index = 1
            second_index=0
            basis_class = Basis(n)
            
            created_symmetric_matrix = basis_class.create_single_symmetric_basis_matrix(first_index = first_index, second_index= second_index)
            
            actual_matrix = 0.5*np.array([[0,1],[1,0]])
            self.assertTrue(np.array_equal(created_symmetric_matrix, actual_matrix))

    def test_create_single_symmetric_matrix_validation_m_smaller_than_n(self):

        n = 4
        first_index = 2
        second_index = 3
        basis_class = Basis(n)

        with self.assertRaises(Exception) as context:
            basis_class.create_single_symmetric_basis_matrix(first_index=first_index, second_index=second_index)
        
        self.assertEqual(str(context.exception), "first_index must be larger than second_index!")

    def test_create_single_symmetric_matrix_validation_m_lowest_value_is_zero(self):

        #maybe this test is redundant since error is caught in create_basis_vectors

        n = 4
        first_index = 3
        second_index = -1
        basis_class = Basis(n)

        with self.assertRaises(Exception) as context:
            basis_class.create_single_symmetric_basis_matrix(first_index=first_index, second_index=second_index)

        self.assertEqual(str(context.exception), "Index can't be less than zero!")

    def test_create_single_symmetric_matrix_validation_n_smaller_than_dimensions(self):
        #Also redunant test since error is caught by python
        n = 4
        first_index = 4
        second_index = 0
        basis_class = Basis(n)
        
        with self.assertRaises(Exception) as context:
            basis_class.create_single_symmetric_basis_matrix(first_index=first_index, second_index=second_index)

        self.assertEqual(str(context.exception), f"index {first_index} is out of bounds for axis 0 with size {n}")

    def test_create_single_antisymmetric_matrix_is_correct(self):   
            n = 2
            first_index = 1
            second_index=0
            basis_class = Basis(n)
            
            created_antisymmetric_matrix = basis_class.create_single_antisymmetric_basis_matrix(first_index = first_index, second_index= second_index)
            
            actual_matrix = 0.5*np.array([[0,-1j],[1j,0]])
            self.assertTrue(np.array_equal(created_antisymmetric_matrix, actual_matrix))

    def test_create_single_antisymmetric_matrix_validation_m_smaller_than_n(self):
        n = 4
        first_index = 2
        second_index = 3
        basis_class = Basis(n)

        with self.assertRaises(Exception) as context:
            basis_class.create_single_antisymmetric_basis_matrix(first_index=first_index, second_index=second_index)
        
        self.assertEqual(str(context.exception), "first_index must be larger than second_index!")

    def test_create_diagonal_matrices_is_correct(self):
        #note this is only checking the su(2) cases need to check su(4) cases, but will do that when doing testing if the full basis is produced.       
        n=2 
        index = 1 
        basis_class = Basis(n)
        expected_matrix = basis_class.create_single_diagonal_basis_matrix(index = index)
        actual_matrix = 0.5*np.array([[1,0],[0,-1]])
        
        self.assertTrue(np.array_equal(actual_matrix, expected_matrix))

    def test_create_diagonal_matrix_validation_upper_bound(self):

        n = 2
        index =2 
        basis_class = Basis(n)

        with self.assertRaises(Exception) as context:
            basis_class.create_single_diagonal_basis_matrix(index = index)

        self.assertEqual(str(context.exception), f"index {index} is out of bounds for axis 0 with size {n}")

    def test_create_diagonal_matrix_validation_lower_bound(self):

        n = 2
        index = 0
        basis_class = Basis(n)

        with self.assertRaises(Exception) as context:
            basis_class.create_single_diagonal_basis_matrix(index = index)
        
        self.assertEqual(str(context.exception), "Index must be greater than zero!")
    

    def test_create_correct_number_of_symmetric_basis_matrices(self):
        
        n = 6
        basis_class = Basis(n)
        number_of_matrices = n*(n-1)/2

        expected_number = len(basis_class.create_all_symmetric_basis_matrices())

        self.assertEqual(expected_number, number_of_matrices)

    def test_all_symmetric_matric_correct_for_dimension_4(self):

        n = 4
        basis_class = Basis(n)
        
        actual_matrix_list = []
        for i in range(1,n):
            for j in range(i):
                holder = np.zeros((n,n))
                holder[i][j] = 0.5
                holder[j][i] = 0.5
                actual_matrix_list.append(holder)
                
        expect_list = basis_class.create_all_symmetric_basis_matrices()
        
        sorted_actual = sorted(actual_matrix_list, key = lambda x: x.tolist())
        sorted_expected = sorted(expect_list, key = lambda x: x.tolist())

        are_equal = all(np.array_equal(a,b) for a,b in zip(sorted_actual, sorted_expected))
        self.assertTrue(are_equal)

    def test_create_correct_number_of_antisymmetric_basis_matrices(self):
        
        n = 8
        basis_class = Basis(n)
        number_of_matrices = n*(n-1)/2

        expected_number = len(basis_class.create_all_antisymmetric_basis_matrices())

        self.assertEqual(expected_number, number_of_matrices)
    
    def test_all_antisymm_matrices_correct_for_su_4(self):
        n = 4
        basis_class = Basis(n)
        actual_matrix_list = []
        for i in range(1,n):
            for j in range(i):
                holder = np.zeros((n,n), dtype=complex)
                holder[i][j] = 0.5j
                holder[j][i] = -0.5j
                actual_matrix_list.append(holder)
                
        expected_list = basis_class.create_all_symmetric_basis_matrices()
        
        sorted_actual = sorted(actual_matrix_list, key = lambda x: sorted([e.imag for row in x for e in row]))
        sorted_expected = sorted(expected_list, key = lambda x: sorted([e.imag for row in x for e in row]))

        are_equal = all(np.array_equal(a,b) for a,b in zip(sorted_actual, sorted_expected))
        print(are_equal)
        self.assertTrue(are_equal)
        


        
    
    







In [93]:
unittest.main(argv=[''], exit=False)

F..

.................
FAIL: test_all_antisymm_matrices_correct_for_su_4 (__main__.BasisTestCase.test_all_antisymm_matrices_correct_for_su_4)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\waqas\AppData\Local\Temp\ipykernel_26480\566619164.py", line 223, in test_all_antisymm_matrices_correct_for_su_4
    self.assertTrue(are_equal)
AssertionError: False is not true

----------------------------------------------------------------------
Ran 20 tests in 0.056s

FAILED (failures=1)


In [89]:
basis_class = Basis(4)

list_antisymm = basis_class.create_all_antisymmetric_basis_matrices()
sorted_expected = sorted(list_antisymm, key = lambda x: sorted([e.imag for row in x for e in row]))

print(sorted_expected)

[array([[0.-0.j , 0.-0.5j, 0.-0.j , 0.-0.j ],
       [0.+0.5j, 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ]]), array([[0.-0.j , 0.-0.j , 0.-0.5j, 0.-0.j ],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.+0.5j, 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ]]), array([[0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.-0.j , 0.-0.j , 0.-0.5j, 0.-0.j ],
       [0.-0.j , 0.+0.5j, 0.-0.j , 0.-0.j ],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ]]), array([[0.-0.j , 0.-0.j , 0.-0.j , 0.-0.5j],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.+0.5j, 0.-0.j , 0.-0.j , 0.-0.j ]]), array([[0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.5j],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.-0.j , 0.+0.5j, 0.-0.j , 0.-0.j ]]), array([[0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ],
       [0.-0.j , 0.-0.j , 0.-0.j , 0.-0.j ],

In [87]:
n=4
actual_matrix_list = []
for i in range(1,n):
    for j in range(i):
        holder = np.zeros((n,n), dtype=complex)
        holder[i][j] = 0.5j
        holder[j][i] = -0.5j
        actual_matrix_list.append(holder)

sorted_actual = sorted(actual_matrix_list, key = lambda x: sorted([e.imag for row in x for e in row]))

print(sorted_actual)



[array([[ 0.+0.j , -0.-0.5j,  0.+0.j ,  0.+0.j ],
       [ 0.+0.5j,  0.+0.j ,  0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ]]), array([[ 0.+0.j ,  0.+0.j , -0.-0.5j,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ],
       [ 0.+0.5j,  0.+0.j ,  0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ]]), array([[ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j , -0.-0.5j,  0.+0.j ],
       [ 0.+0.j ,  0.+0.5j,  0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ]]), array([[ 0.+0.j ,  0.+0.j ,  0.+0.j , -0.-0.5j],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ],
       [ 0.+0.5j,  0.+0.j ,  0.+0.j ,  0.+0.j ]]), array([[ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j , -0.-0.5j],
       [ 0.+0.j ,  0.+0.j ,  0.+0.j ,  0.+0.j ],
       [ 0.+0.j ,  0.+0.5j,  0.+0.j ,  0.+0.j ]]), array([[ 

In [94]:
for a,b in zip(sorted_expected, sorted_actual):
    print(np.array_equal(a,b))

True
True
True
True
True
True


In [95]:
all(np.array_equal(a,b) for a,b in zip(sorted_actual, sorted_expected))

True